**卷积神经网络 (Convolutional Nerual Network, CNN)**

@ Date: 2025-04-06<br>
@ Author: Rui Zhu<br>
@ Note: <br> 
    1. CNN是一类强大的、为处理图像数据而设计的神经网络<br>
    2. CNN需要的参数少于全连接架构的网络, 而且卷积容易使用GPU平行计算

In [1]:
import torch
from torch import nn
import matplotlib.pyplot as plt

---
# 基本概念
- 图像中有丰富的结构, CNN通过卷积层提取了图像的结构特征
- 图像中的结构特性:
    1. 平移不变性(Translation Invariance): 图像中的物体可以出现在任意位置, 识别物体不应关注其在图像中的具体位置
    2. 局部性(Locality): 图像中的对象识别关注的是局部区域, 而不是整个图像
- 从MLP到CNN(数学表示):
    1. 已知二维输入图像$X_{i, j}$, 则MLP第一个隐藏层(特征图)可表示为:$$H_{i, j} = U_{i, j} + \sum_k \sum_l W_{i, j}^{k, l} X_{k, l}$$
    2. 使用($k=i+a$, $l=j+b$)重新索引下标($k, l$): $$H_{i, j} = U_{i, j} + \sum_a \sum_b V_{i, j}^{a, b} X_{i+a, j+b}$$
    3. 由平移不变性, 检测对象在输入图像中的平移反映在特征图中的平移, 即$U_{i, j}$和$V_{i, j}^{a, b}$不依赖于$(i, j)$: $$H_{i, j} = u + \sum_a \sum_b V^{a, b} X_{i+a, j+b}$$
    Note:
       - 这就是卷积的数学表达, $V^{a, b}$称为卷积核, 即卷积层的权重, 是可学习的参数
       - $V^{a, b}$比$V_{i, j}^{a, b}$大幅缩减参数规模, 通过对象在图像中的平移不变性
    4. 由局部性原则, 卷积区域比图像小, 由此可以继续改写为: $$H_{i, j} = u + \sum_{a=-\Delta}^{\Delta} \sum_{b=-\Delta}^{\Delta} V^{a, b} X_{i+a, j+b}$$
    Note:
        - 其中$\Delta$是卷积核的半宽, 即5x5的卷积核, 半宽为2
- 卷积与互相关
    1. 数学中的卷积: $$(f*g)(x) = \int f(z)g(x-z)dz$$
        (对于图像离散化)$$(f*g)(i, j) = \sum_a \sum_b f(a, b)g(i-a, j-b)$$
    2. 数学中的互相关: $$(f*g)(x) = \int f(z)g(x+z)dz$$
        (对于图像离散化)$$(f*g)(i, j) = \sum_a \sum_b f(a, b)g(i+a, j+b)$$
    3. 卷积和互相关是非常类似的操作, 差别在于是否反转kernal
    4. CNN中使用的操作实际上是计算互相关
- 通道(channel): 对于图像, 通道表示颜色信息的维度. 对于RGB图像, 通道数为3; 灰度图像, 通道数为1

---
# 卷积层
- 严格来讲, 卷积层是错误的叫法, 实际表达的运算是互相关
- 但两者差别只在于卷积需要水平和垂直翻转二维卷积核张量
- 由于卷积核是从数据中学习得到的, 因此无论采用卷积还是互相关, 卷积层的输出不会受到影响
- 在卷积层中, 输入张量和核张量通过互相关运算, 然后添加偏置标量生成输出张量
- Feature Map: 卷积层也称特征映射
- 元素: 卷积核张量上的每一个权重称为元素
- 感受野(receptive field): 对于某一层的任意元素x, 其感受野指在向前传播期间可能影响x计算的所有元素(来自所有之前层)

## 定义互相关操作

In [2]:
def coor2d(X, K):
    """
    定义互相关运算
    """
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = torch.sum(X[i:i + h, j:j + w] * K)
    return Y

X = torch.tensor([
    [0, 1, 2], 
    [3, 4, 5], 
    [6, 7, 8]
])
K = torch.tensor([
    [0, 1],
    [2, 3]
])
Y = coor2d(X, K)
print(Y)

tensor([[19., 25.],
        [37., 43.]])


## 定义卷积层
- 卷积层的两个被训练参数: 卷积核, 标量偏置
- 高度h和宽度w的卷积核称为hxw卷积核, 带有hxw卷积核的卷积层, 称为hxw卷积层

In [3]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self, x):
        return coor2d(x, self.weight) + self.bias

## 卷积层的应用举例: 边缘检测

In [4]:
# 定义测试图像
X = torch.ones((6, 8))
X[:, 2:6] = 0
print(X)

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])


In [5]:
# 构造卷积核, 水平两个元素相同输出0, 否则非0
K = torch.tensor([[1, -1]])

In [6]:
Y = coor2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [8]:
coor2d(X.T, K)  # 这个卷积核只能检测垂直边缘, 不能检测水平边缘

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

## 学习卷积核
- 已知输入X和输出Y, 通过训练, 学习出这个卷积核

In [9]:
# 构造卷积层
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)

# 使用4维输入和输出格式(批量大小, 通道, 高度, 宽度)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 0.03

# 训练卷积层
for i in range(20):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    loss.sum().backward()
    
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    print(f"epoch {i + 1}, loss {loss.sum():.3f}")

print(f"Final kernel: {conv2d.weight.data.reshape(1, 2)}")

epoch 1, loss 6.078
epoch 2, loss 3.033
epoch 3, loss 1.590
epoch 4, loss 0.874
epoch 5, loss 0.500
epoch 6, loss 0.296
epoch 7, loss 0.180
epoch 8, loss 0.111
epoch 9, loss 0.069
epoch 10, loss 0.044
epoch 11, loss 0.028
epoch 12, loss 0.018
epoch 13, loss 0.011
epoch 14, loss 0.007
epoch 15, loss 0.005
epoch 16, loss 0.003
epoch 17, loss 0.002
epoch 18, loss 0.001
epoch 19, loss 0.001
epoch 20, loss 0.000
Final kernel: tensor([[ 0.9976, -1.0022]])


---
# 填充和步幅
- 填充(padding): 解决卷积丢失图像边缘信息的有效方法
- 步幅(stride): 原始图像的分辨率过高, 希望大幅减少图像的高度和宽度时采用的方法
- 卷积的输出形状取决于输入形状和卷积核形状
- 假设输入形状$n_h \times n_w$, 卷积核形状$k_n \times k_w$, 则卷积后的输出形状为($n_h - k_n + 1$, $n_w - k_w + 1$)
- 很少使用不一致的步幅和填充

## 填充
- 只有使用1x1的卷积核, 卷积后的形状才与原图像形状相同
- 卷积核约大, 丢失像素越多
- 连续卷积多次后, 累积丢失的像素数就会增多
- 填充: 在输入图像的边缘填充0, 假设填充$p_h$行, $p_w$列, 则输出形状($n_h + p_h - k_n + 1$, $n_w + p_w - k_w + 1$)
- 通常使用`奇数`的卷积核大小
- 当卷积核或输入图像的形状不是正方形时, 可以填充不同的数值, 使输出形状是正方形

In [13]:
def comp_conv2d(conv2d, X):
    X = X.reshape((1, 1) + X.shape)  # 这里的(1, 1)表示批量大小和通道数
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:])

conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1)
X = torch.rand(size=(8, 8))
comp_conv2d(conv2d, X).shape  # 输出形状为(8, 8), 因为padding=1

torch.Size([8, 8])

In [14]:
conv2d = nn.Conv2d(1, 1, kernel_size=(5, 3), padding=(2, 1))
comp_conv2d(conv2d, X).shape  # 输出形状为(8, 8), 因为padding=(2, 1)

torch.Size([8, 8])

## 步幅
- 计算互相关时, 卷积窗口默认向下、向右滑动1个元素
- 为了高效计算或缩减采样次数, 卷积窗口可以跳过一些元素, 每次滑动的元素数量称为步幅

In [16]:
conv2d = nn.Conv2d(1, 1, kernel_size=3, padding=1, stride=2)
comp_conv2d(conv2d, X).shape  # 输出形状为(4, 4), 因为stride=2

torch.Size([4, 4])

In [17]:
conv2d = nn.Conv2d(1, 1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
comp_conv2d(conv2d, X).shape  # 输出形状为(3, 2), 因为stride=(3, 4)

torch.Size([2, 2])

---
# 多输入多输出通道